#Controlar la presencia de oligosacáridos ramificados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Biopython
!pip install pandarallel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.7 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=a3f4f920e0cf94d5f7fe0ec93281ef2c0fd9a5c52bad31b92868eb25e2e99955
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel


In [ ]:
#@title Importar archivos

import pandas as pd
import os

input_folder = "/content/drive/MyDrive/TFM/T1/oligosacaridos_ramificados"
input_file = os.path.join(input_folder,"df_harm_filtro_3.csv")
df_harm_1 = pd.read_csv(input_file, sep = ',')

In [ ]:
#@title Función para comprobar si hay o no oligosacáridos ramificados

def check_oligosacaridos(row):
    entrada = row['PDB_entry_id']
    if entrada[0] in ['3']:
      subfolder = entrada[0]
      cif_file = os.path.join(input_folder, subfolder, f"{entrada.lower()}.cif")

      try:
          cif_dict = MMCIF2Dict(cif_file)
          if "_pdbx_entity_branch.type" in cif_dict:
              return True  # Contiene oligosacárido ramificado
      except Exception as e:
          print(f"Error con {entrada}: {e}")
          entrada_no_encontrada.append(entrada)

      return False

In [ ]:
#@title Dataframe con las entradas a analizar

df_lista_3 = df_harm_1[df_harm_1['PDB_entry_id'].str.startswith('3')].copy()
print(len(df_lista_3))

9068


In [ ]:
#@title Aplicar la función al dataframe y crear el dataframe de output con solo los oligosacaridos ramificados

# Inicializar pandarallel (ajusta workers si tienes más núcleos)
pandarallel.initialize(nb_workers=2, progress_bar=True)

df_lista_3['es_oligosacarido_ramificado'] = df_lista_3.parallel_apply(check_oligosacaridos, axis=1)

# Evitar error de NaN
df_lista_3['es_oligosacarido_ramificado'] = df_lista_3['es_oligosacarido_ramificado'].fillna(False)

df_oligos = df_lista_3[df_lista_3['es_oligosacarido_ramificado']]
print(f"Total encontrados: {len(df_oligos)}")

output_file = os.path.join(input_folder, "oligosacaridos_ramificados.csv")
df_oligos.to_csv(output_file, index=False)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandarallel/core.py", line 95, in __call__
    result = self.work_function(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, i

KeyboardInterrupt: 

  File "/usr/local/lib/python3.11/dist-packages/pandas/core/apply.py", line 916, in apply
    return self.apply_standard()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/Bio/PDB/MMCIF2Dict.py", line 40, in __init__
    for token in tokens:
